## ¡Bienvenido de nuevo a los cuadernos de Python!

¿Me extrañabas?

### Y bienvenido a la Semana 4, Día 2 - ¡Presentación de Langgraph!

In [1]:
from typing import Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
import random


In [2]:
# Algunas constantes útiles

nouns = ["Coles", "Unicornios", "Tostadora", "Pinguinos", "Bananas", "Zombies", "Arcoiris", "Anguilas", "Pepinillos", "Muffins"]
adjectives = ["malo", "pedante", "existencial", "malhumorado", "brillante", "desconfiado", "sarcástico", "molloso", "misterioso"]

# Algunas funciones útiles

def get_random_noun():
    return random.choice(nouns)

def get_random_adjective():
    return random.choice(adjectives)

In [ ]:
# Nuestro primer paso favorito! Crew lo hacía por nosotros, por cierto.
load_dotenv(override=True)

In [ ]:
def shout(text: Annotated[str, "algo para gritar"]) -> str:
    print(text.upper())
    return text.upper()

shout("hola")

### Una palabra sobre "anotado"

Probablemente sepas esto; Tipo de insinción es una característica en Python que le permite especificar el tipo de algo:

`my_favorite_things: list`

Pero es posible que no sepas esto:

También puede usar algo llamado "anotado" para agregar información adicional que alguien más podría encontrar útil:

`my_favorite_things: anotado [lista," estos son algunos de los míos "]`

Langgraph necesita que usemos esta función cuando definimos nuestro objeto de estado.

Quiere que le digamos qué función debe llamar para actualizar el estado con un nuevo valor.

Esta función se llama **reductor**.

Langgraph proporciona un reductor predeterminado llamado `add_messages` que se encarga del caso más común.

Y con suerte explica por qué el estado se ve así.

### Paso 1: Definir el objeto de estado

Puedes usar cualquier objeto Python; Pero es más común usar un tipEddict o un basemodelo pydantic.

In [5]:

class State(BaseModel):
        
    messages: Annotated[list, add_messages]


### Paso 2: Iniciar el Graph Builder con esta clase State

In [6]:
graph_builder = StateGraph(State)

### Paso 3: Crear un nodo

Un nodo puede ser cualquier función de Python.

El reductor que establecemos antes se llama automáticamente para combinar esta respuesta con respuestas anteriores

In [ ]:
def our_first_node(old_state: State) -> State:

    reply = f"{random.choice(nouns)} son {random.choice(adjectives)}"
    messages = [{"role": "assistant", "content": reply}]

    new_state = State(messages=messages)

    return new_state

graph_builder.add_node("first_node", our_first_node)

### Paso 4: Creamos las aristas

In [ ]:
graph_builder.add_edge(START, "first_node")
graph_builder.add_edge("first_node", END)

### Paso 5: Compilamos el Grafo

In [9]:
graph = graph_builder.compile()

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

### Listo! El momento de la verdad!

In [ ]:
def chat(user_input: str, history):
    message = {"role": "user", "content": user_input}
    messages = [message]
    state = State(messages=messages)
    result = graph.invoke(state)
    print(result)
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()

### Por cierto, ¡por qué te mostré eso?

Para señalar que Langgraph se trata de funciones de Python, ¡no necesita involucrar a LLM!

Ahora volveremos a hacer los 5 pasos, pero en 1 disparo:

In [12]:
# Paso 1: Define el objeto de estado

class State(BaseModel):
    messages: Annotated[list, add_messages]


In [13]:
# Paso 2: Iniciar el Graph Builder con esta clase State
graph_builder = StateGraph(State)

In [ ]:
# Paso 3: Crear un nodo

llm = ChatOpenAI(model="gpt-4o-mini")

def chatbot_node(old_state: State) -> State:
    response = llm.invoke(old_state.messages)
    new_state = State(messages=[response])
    return new_state

graph_builder.add_node("chatbot", chatbot_node)

In [ ]:
# Paso 4: Crear aristas
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

In [ ]:
# Paso 5: Compilar el Grafo
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

### ¡Y esto es todo! Hagamos esto:

In [ ]:
def chat(user_input: str, history):
    initial_state = State(messages=[{"role": "user", "content": user_input}])
    result = graph.invoke(initial_state)
    print(result)
    return result['messages'][-1].content


gr.ChatInterface(chat, type="messages").launch()